# Assistant


Azure OpenAI Assistants를 사용하면 사용자 지정 지침과 코드 인터프리터, 사용자 지정 함수와 같은 고급 도구를 통해 필요에 맞는 AI 어시스턴트를 만들 수 있습니다. 이 문서에서는 Assistants API를 시작하는 방법에 대한 심층적인 가이드를 제공합니다.


## API 버전: 2024-02-15-preview
사용 가능 여부 확인: 

https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models#assistants-preview

## 어시스턴트 생성

In [31]:
import os
import json
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv(override=True)
    
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2025-03-01-preview",
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"),
)

# Create an assistant
assistant = client.beta.assistants.create(
    name="Data Visualization",
    instructions=
    """
        당신은 데이터 기반으로 흥미로운 시각화를 만드는 유용한 AI 어시스턴트입니다.
        코드를 작성하고 테스트할 수 있는 샌드박스 환경에 접근할 수 있습니다.
        시각화 생성을 요청받으면 다음 단계를 따르세요:
        1. 코드를 작성하세요.
        2. 새로운 코드를 작성할 때마다 미리보기를 표시하여 작업 과정을 보여주세요.
        3. 코드를 실행하여 정상 동작하는지 확인하세요.
        4. 코드가 성공적으로 실행되면 시각화를 표시하세요.
        5. 코드 실행에 실패하면 오류 메시지를 표시하고, 코드를 수정하여 위 단계를 반복하세요.
    """,
    tools=[{"type": "code_interpreter"}],
    model = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
)

In [9]:
print(assistant.model_dump_json(indent=2))

{
  "id": "asst_pBSvgA9ZQ8kPpt6dwLHwEBQm",
  "created_at": 1746671759,
  "description": null,
  "instructions": "\n        당신은 데이터 기반으로 흥미로운 시각화를 만드는 유용한 AI 어시스턴트입니다.\n        코드를 작성하고 테스트할 수 있는 샌드박스 환경에 접근할 수 있습니다.\n        시각화 생성을 요청받으면 다음 단계를 따르세요:\n        1. 코드를 작성하세요.\n        2. 새로운 코드를 작성할 때마다 미리보기를 표시하여 작업 과정을 보여주세요.\n        3. 코드를 실행하여 정상 동작하는지 확인하세요.\n        4. 코드가 성공적으로 실행되면 시각화를 표시하세요.\n        5. 코드 실행에 실패하면 오류 메시지를 표시하고, 코드를 수정하여 위 단계를 반복하세요.\n    ",
  "metadata": {},
  "model": "gpt-4",
  "name": "Data Visualization",
  "object": "assistant",
  "tools": [
    {
      "type": "code_interpreter"
    }
  ],
  "response_format": "auto",
  "temperature": 1.0,
  "tool_resources": {
    "code_interpreter": {
      "file_ids": []
    },
    "file_search": null
  },
  "top_p": 1.0
}


## 스레드 생성

스레드는 어시스턴트와 사용자 간의 대화 세션입니다. 스레드는 메시지를 저장하며 모델의 컨텍스트에 맞게 내용을 자동으로 잘라냅니다.

In [10]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_vM9xTzH3FygdlwKdjiipp7U9', created_at=1746671777, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


In [11]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="사인파 시각화 만들기"
)

In [12]:
# list thread messages
thread_messages = client.beta.threads.messages.list(thread.id)
print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_HpcI79yL4V0tobf87NLmPXj2",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "사인파 시각화 만들기"
          },
          "type": "text"
        }
      ],
      "created_at": 1746671795,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_vM9xTzH3FygdlwKdjiipp7U9"
    }
  ],
  "object": "list",
  "first_id": "msg_HpcI79yL4V0tobf87NLmPXj2",
  "last_id": "msg_HpcI79yL4V0tobf87NLmPXj2",
  "has_more": false
}


In [13]:
# create a thread
# you can also use intructions here to override the default instructions
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [15]:
# retrieve thread status
# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

in_progress


In [28]:
# check the status of the run
import time
from IPython.display import clear_output

start_time = time.time()

status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    time.sleep(5)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = run.status
    print(f'Status: {status}')
    clear_output(wait=True)

messages = client.beta.threads.messages.list(
  thread_id=thread.id
) 

print(f'Status: {status}')
print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
print(messages.model_dump_json(indent=2))

Status: completed
Elapsed time: 0 minutes 7 seconds
{
  "data": [
    {
      "id": "msg_lncGKrBxQ0OdKfFZwX4xAlec",
      "assistant_id": "asst_pBSvgA9ZQ8kPpt6dwLHwEBQm",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "아래는 사인파를 생성하고 시각화하는 데 사용한 코드입니다:\n\n```python\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Generate data for the sine wave\nx = np.linspace(0, 2 * np.pi, 1000)\ny = np.sin(x)\n\n# Plot the sine wave\nplt.figure(figsize=(10, 5))\nplt.plot(x, y, label='Sine wave')\nplt.title(\"Sine Wave\")\nplt.xlabel(\"x\")\nplt.ylabel(\"sin(x)\")\nplt.grid(True)\nplt.legend()\nplt.show()\n```\n\n이 코드는 `numpy` 라이브러리를 사용하여 `0`에서 \\(2\\pi\\)까지 균등하게 분포된 1000개의 점을 포함하는 배열 `x`를 생성합니다. 그런 다음 사인 함수를 사용하여 각 점에 대한 값을 계산하여 배열 `y`에 저장합니다. 마지막으로 `matplotlib` 라이브러리를 사용하여 사인파를 그립니다."
          },
          "type": "text"
        }
      ],
      "created_at": 1746671867,
      "incomp

In [17]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_8Dlp2c7zHqpwp9DS8EisOoSO",
      "assistant_id": "asst_pBSvgA9ZQ8kPpt6dwLHwEBQm",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "image_file": {
            "file_id": "assistant-BhkJ7ZfoHVxrdtAE9JB7Qu",
            "detail": null
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "위의 시각화는 \\( y = \\sin(x) \\)의 사인파를 나타낸 것입니다. x축은 \\( 0 \\)에서 \\( 2\\pi \\)까지의 값을, y축은 사인 함수에 의한 결과 값을 나타냅니다. 그리드는 사인파의 주기와 진폭을 쉽게 볼 수 있도록 설정되어 있습니다. 다른 시각화나 추가적인 설명이 필요하면 말씀해 주세요!"
          },
          "type": "text"
        }
      ],
      "created_at": 1746671815,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_XGTPnOWVEpdGVSifhdunJkl8",
      "status": null,
      "thread_id": "thread_vM9xTzH3FygdlwKdjiipp7U9"
    },
   

In [18]:
# show the file name
data = json.loads(messages.model_dump_json(indent=2))  # Load JSON data into a Python object
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)  # Outputs: assistant-1YGVTvNzc2JXajI5JU9F0HMD

assistant-BhkJ7ZfoHVxrdtAE9JB7Qu


In [29]:
# download the file
content = client.files.content(image_file_id)

image= content.write_to_file("sinewave.png")

In [20]:
# rewrite the code to display the image in jupyter notebook

from PIL import Image

# Display the image in the default image viewer
image = Image.open("sinewave.png")
image.show()

## 후속 질문하기

In [21]:
# Add a new user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="사인파를 생성하는 데 사용한 생성된 코드를 보여주세요."
)

In [22]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions  but these will override the default instructions
)

# Retrieve the status of the run
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

status = run.status
print(status)

in_progress


In [25]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_lncGKrBxQ0OdKfFZwX4xAlec",
      "assistant_id": "asst_pBSvgA9ZQ8kPpt6dwLHwEBQm",
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "아래는 사인파를 생성하고 시각화하는 데 사용한 코드입니다:\n\n```python\nimport numpy as np\nimport matplotlib.pyplot as plt\n\n# Generate data for the sine wave\nx = np.linspace(0, 2 * np.pi, 1000)\ny = np.sin(x)\n\n# Plot the sine wave\nplt.figure(figsize=(10, 5))\nplt.plot(x, y, label='Sine wave')\nplt.title(\"Sine Wave\")\nplt.xlabel(\"x\")\nplt.ylabel(\"sin(x)\")\nplt.grid(True)\nplt.legend()\nplt.show()\n```\n\n이 코드는 `numpy` 라이브러리를 사용하여 `0`에서 \\(2\\pi\\)까지 균등하게 분포된 1000개의 점을 포함하는 배열 `x`를 생성합니다. 그런 다음 사인 함수를 사용하여 각 점에 대한 값을 계산하여 배열 `y`에 저장합니다. 마지막으로 `matplotlib` 라이브러리를 사용하여 사인파를 그립니다."
          },
          "type": "text"
        }
      ],
      "created_at": 1746671867,
      "incomplete_at": null,
      "incomplete_details": null,
  

In [27]:
# Print the code
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

아래는 사인파를 생성하고 시각화하는 데 사용한 코드입니다:

```python
import numpy as np
import matplotlib.pyplot as plt

# Generate data for the sine wave
x = np.linspace(0, 2 * np.pi, 1000)
y = np.sin(x)

# Plot the sine wave
plt.figure(figsize=(10, 5))
plt.plot(x, y, label='Sine wave')
plt.title("Sine Wave")
plt.xlabel("x")
plt.ylabel("sin(x)")
plt.grid(True)
plt.legend()
plt.show()
```

이 코드는 `numpy` 라이브러리를 사용하여 `0`에서 \(2\pi\)까지 균등하게 분포된 1000개의 점을 포함하는 배열 `x`를 생성합니다. 그런 다음 사인 함수를 사용하여 각 점에 대한 값을 계산하여 배열 `y`에 저장합니다. 마지막으로 `matplotlib` 라이브러리를 사용하여 사인파를 그립니다.
